#### Lesson 3: Reflection and Blogpost Writing

#### Setup

In [1]:
#r "nuget:AutoGen,0.2.0"
#load "util.csx"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using System.Threading;
using OpenAI;
using OpenAI.Chat;

var chatClient = ChatClientProvider.Create("gpt-4o-mini");

Installed Packages AutoGen, 0.2.0

The Task!

In [2]:
var task = """
    Write a concise but engaging blogpost about
    DeepLearning.AI. Make sure the blogpost is
    within 100 words.
    """;

Create a writer agent

In [3]:
var writer = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Writer",
    systemMessage: """
    You are a writer. You write engaging and concise blogpost (with title) on given topics.
    You must polish your writing based on the feedback you receive and give a refined version.
    Only return your final work without additional comments."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

var reply = await writer.SendAsync(task);

from: Writer
**Unlocking AI Potential: DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals and organizations to harness the power of deep learning. Founded by AI pioneer Andrew Ng, this platform offers a range of online courses, specializations, and resources tailored for both beginners and experienced practitioners. With a hands-on approach, learners dive into neural networks, data science, and real-world applications, bridging the gap between theory and practice. As AI continues to reshape industries, DeepLearning.AI equips the next generation of innovators with the skills needed to thrive in this dynamic landscape. Join the AI revolution and elevate your expertise today!



Adding reflection by creating a critic agent to reflect on the work of the writer agent.

In [4]:
var critic = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Critic",
    systemMessage: """
    You are a critic. You review the work of the writer and provide constructive feedback to help improve the quality of the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

var conversation = await critic.SendAsync(
    receiver: writer,
    message: task,
    maxRound: 3)
    .ToListAsync();

// Use the last message in the conversation as the final result
var res = conversation.Last();

from: Writer
**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is revolutionizing the landscape of artificial intelligence education. Founded by AI pioneer Andrew Ng, this platform offers comprehensive courses that empower learners to master deep learning techniques. With a focus on practical applications, students gain hands-on experience through projects and real-world case studies. The community-driven approach fosters collaboration and innovation, connecting aspiring AI practitioners worldwide. Whether you're a beginner or an experienced professional, DeepLearning.AI provides the tools and knowledge to thrive in this dynamic field. Dive into the world of AI today and unlock your potential with DeepLearning.AI!

from: Critic
**Feedback:**

Your blog post effectively highlights the key features of DeepLearning.AI, and the enthusiasm for the platform is palpable. However, to enhance clarity and engagement, consider these suggestions:

1. **Hook:** Start with a captivating

#### Nested chat using middleware pattern
Python AutoGen allows you to create a Json object task list and pass it to agent using `register_nested_chats`
This is not supported in AutoGen.Net.
But you can achieve the same using the middleware pattern.

In [5]:
var SEOReviewer = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "SEO_Reviewer",
    systemMessage: """
    You are an SEO reviewer, known for your ability to optimize content for search engines, ensuring that it ranks well and attracts organic traffic.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [6]:
var LegalReviewer = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Legal_Reviewer",
    systemMessage: """
    You are a legal reviewer, known for your ability to ensure that content is legally compliant and free from any potential legal issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [7]:
var EthicsReviewer = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Ethics_Reviewer",
    systemMessage: """
    You are an ethics reviewer, known for your ability to ensure that content is ethically sound and free from any potential ethical issues.
    Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.
    Begin the review by stating your role.
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

In [8]:
var MetaReviewer = new OpenAIChatAgent(
    chatClient: chatClient,
    name: "Meta_Reviewer",
    systemMessage: """
    You are a meta reviewer, you aggragate and review the work of other reviewers and give a final suggestion on the content."
    """)
    .RegisterMessageConnector()
    .RegisterPrintMessage();

Define nest chat middleware

In [9]:
class NestChatMiddleware : IMiddleware
{
    private readonly IAgent seo;
    private readonly IAgent legal;
    private readonly IAgent ethics;
    private readonly IAgent meta;
    private readonly IAgent writer;

    public NestChatMiddleware(IAgent writer, IAgent seo, IAgent legal, IAgent ethics, IAgent meta)
    {
        this.writer = writer;
        this.seo = seo;
        this.legal = legal;
        this.ethics = ethics;
        this.meta = meta;
    }

    public string? Name => nameof(NestChatMiddleware);

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent critic, CancellationToken cancellationToken = default)
    {
        // trigger only when the last message is from writer
        if (context.Messages.Last().From != writer.Name)
        {
            return await critic.GenerateReplyAsync(context.Messages, context.Options, cancellationToken);
        }

        var messageToReview = context.Messages.Last();
        var reviewPrompt = $"""
            Review the following content.

            {messageToReview.GetContent()}
            """;
        // SEO Review
        var seoReview = await critic.SendAsync(
            receiver: seo,
            message: reviewPrompt,
            maxRound: 1)
            .ToListAsync();
        var legalReview = await critic.SendAsync(
            receiver: legal,
            message: reviewPrompt,
            maxRound: 1)
            .ToListAsync();

        var ethicsReview = await critic.SendAsync(
            receiver: ethics,
            message: reviewPrompt,
            maxRound: 1)
            .ToListAsync();

        var reviews = seoReview.Concat(legalReview).Concat(ethicsReview);

        var metaReview = await critic.SendAsync(
            receiver: meta,
            message: "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
            chatHistory: reviews,
            maxRound: 1)
            .ToListAsync();

        var lastReview = metaReview.Last();
        lastReview.From = critic.Name;

        return lastReview;
    }
}

In [10]:
// Orchestrate the nested chats to solve the task
var middleware = new NestChatMiddleware(
    writer: writer,
    seo: SEOReviewer,
    legal: LegalReviewer,
    ethics: EthicsReviewer,
    meta: MetaReviewer);

In [11]:
var nestChatCritic = critic
    .RegisterMiddleware(middleware)
    .RegisterPrintMessage();

In [12]:
conversation = await nestChatCritic.SendAsync(
    message: task,
    receiver: writer,
    maxRound: 3)
    .ToListAsync();

var finalResult = conversation.Last();

from: Writer
**Unlocking the Future with DeepLearning.AI**

DeepLearning.AI is at the forefront of artificial intelligence education, empowering individuals and organizations with essential skills in deep learning and machine learning. Founded by AI pioneer Andrew Ng, it offers comprehensive courses that simplify complex concepts, making them accessible to all. With a focus on practical applications, learners can engage in hands-on projects that enhance their understanding and bolster their resumes. As AI continues to shape industries worldwide, DeepLearning.AI equips the next generation of innovators with the tools they need to thrive in a data-driven world. Embrace the AI revolution and elevate your career today!

from: SEO_Reviewer
As an SEO reviewer, here are my suggestions to optimize the content for better search engine visibility:

- **Incorporate Target Keywords**: Include relevant keywords such as "artificial intelligence courses," "deep learning training," and "machine learni